   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Actividad 9: Árbol de Decisión - Detección de Phishing\n",
    "\n",
    "**Objetivo:** Implementar un modelo de aprendizaje supervisado usando árbol de decisión para clasificar si un correo electrónico o mensaje SMS es **Phishing o Legítimo**.\n",
    "\n",
    "## Variables del Modelo\n",
    "\n",
    "### Variables Independientes (Predictoras):\n",
    "1. **Remitente Sospechoso** (sender_suspicious): ¿El remitente parece sospechoso? (0-10)\n",
    "2. **Contiene URL** (contains_url): ¿El mensaje contiene enlaces? (0=No, 1=Sí)\n",
    "3. **Dominio Sospechoso** (domain_suspicious): ¿El dominio de la URL es sospechoso? (0-10)\n",
    "4. **Urgencia/Amenaza** (urgency_tone): ¿Usa tono de urgencia o amenaza? (0-10)\n",
    "5. **Solicita Información** (requests_info): ¿Pide información personal/contraseñas? (0-10)\n",
    "6. **Errores Gramaticales** (grammar_errors): Cantidad de errores gramaticales u ortográficos (0-10)\n",
    "7. **Oferta Irreal** (unrealistic_offer): ¿Promete premios/ofertas increíbles? (0-10)\n",
    "\n",
    "### Variable Dependiente (Objetivo):\n",
    "- **Es Phishing** (is_phishing): 1 = Phishing, 0 = Legítimo"
   ]

## 1. Importación de Librerías

In [ ]:
# Instalación de librerías necesarias (ejecutar solo una vez)
# !pip install pandas numpy scikit-learn matplotlib seaborn graphviz pydotplus fpdf

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
np.random.seed(42)

print("✓ Librerías importadas correctamente")

: 

   "source": [
    "## 2. Generación del Dataset\n",
    "\n",
    "Vamos a generar un dataset sintético de 1200 mensajes (emails y SMS) con características realistas de mensajes de phishing y legítimos."
   ]

In [ ]:
   "source": [
    "# Configuración del generador de números aleatorios\n",
    "np.random.seed(42)\n",
    "\n",
    "# Número de registros\n",
    "n_records = 1200\n",
    "\n",
    "# Generar dos grupos: legítimos (60%) y phishing (40%)\n",
    "n_legitimate = int(n_records * 0.6)\n",
    "n_phishing = n_records - n_legitimate\n",
    "\n",
    "# Mensajes LEGÍTIMOS (puntuaciones bajas en indicadores de phishing)\n",
    "legitimate_data = {\n",
    "    'remitente_sospechoso': np.random.normal(2, 1.5, n_legitimate).clip(0, 10),\n",
    "    'contiene_url': np.random.choice([0, 1], n_legitimate, p=[0.4, 0.6]),\n",
    "    'dominio_sospechoso': np.random.normal(1.5, 1.2, n_legitimate).clip(0, 10),\n",
    "    'tono_urgencia': np.random.normal(2, 1.5, n_legitimate).clip(0, 10),\n",
    "    'solicita_info': np.random.normal(1, 1.2, n_legitimate).clip(0, 10),\n",
    "    'errores_gramaticales': np.random.normal(1, 1, n_legitimate).clip(0, 10),\n",
    "    'oferta_irreal': np.random.normal(0.5, 0.8, n_legitimate).clip(0, 10)\n",
    "}\n",
    "\n",
    "# Mensajes PHISHING (puntuaciones altas en indicadores de phishing)\n",
    "phishing_data = {\n",
    "    'remitente_sospechoso': np.random.normal(7, 2, n_phishing).clip(0, 10),\n",
    "    'contiene_url': np.random.choice([0, 1], n_phishing, p=[0.1, 0.9]),\n",
    "    'dominio_sospechoso': np.random.normal(7.5, 1.8, n_phishing).clip(0, 10),\n",
    "    'tono_urgencia': np.random.normal(7.5, 1.5, n_phishing).clip(0, 10),\n",
    "    'solicita_info': np.random.normal(7, 2, n_phishing).clip(0, 10),\n",
    "    'errores_gramaticales': np.random.normal(6, 2, n_phishing).clip(0, 10),\n",
    "    'oferta_irreal': np.random.normal(6.5, 2, n_phishing).clip(0, 10)\n",
    "}\n",
    "\n",
    "# Crear DataFrames separados\n",
    "df_legitimate = pd.DataFrame(legitimate_data)\n",
    "df_legitimate['es_phishing'] = 0\n",
    "\n",
    "df_phishing = pd.DataFrame(phishing_data)\n",
    "df_phishing['es_phishing'] = 1\n",
    "\n",
    "# Combinar y mezclar\n",
    "df = pd.concat([df_legitimate, df_phishing], ignore_index=True)\n",
    "df = df.sample(frac=1, random_state=42).reset_index(drop=True)\n",
    "\n",
    "# Redondear valores\n",
    "for col in df.columns:\n",
    "    if col != 'es_phishing':\n",
    "        df[col] = df[col].round(1)\n",
    "\n",
    "print(f\"✓ Dataset generado con {len(df)} mensajes\")\n",
    "print(f\"\\nDistribución de la variable objetivo:\")\n",
    "print(df['es_phishing'].value_counts())\n",
    "print(f\"\\n  • Mensajes legítimos: {(df['es_phishing']==0).sum()} ({(df['es_phishing']==0).mean()*100:.1f}%)\")\n",
    "print(f\"  • Mensajes de phishing: {(df['es_phishing']==1).sum()} ({(df['es_phishing']==1).mean()*100:.1f}%)\")"
   ]

In [ ]:
   "source": [
    "# Visualizar primeros registros\n",
    "print(\"\\n📧 Primeros 10 mensajes del dataset:\\n\")\n",
    "df.head(10)"
   ]

In [ ]:
# Estadísticas descriptivas
print("\n📈 Estadísticas descriptivas del dataset:\n")
df.describe()

In [ ]:
# Guardar dataset en CSV
df.to_csv('dataset_creditos.csv', index=False)
print("✓ Dataset guardado en 'dataset_creditos.csv'")

## 3. Análisis Exploratorio de Datos (EDA)

In [ ]:
# Crear visualizaciones del dataset
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Análisis Exploratorio del Dataset de Créditos', fontsize=16, fontweight='bold')

# Distribución de ingresos
axes[0, 0].hist(df['ingreso_mensual'], bins=30, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Distribución de Ingreso Mensual')
axes[0, 0].set_xlabel('Ingreso Mensual ($)')
axes[0, 0].set_ylabel('Frecuencia')

# Distribución de puntuación crediticia
axes[0, 1].hist(df['puntuacion_crediticia'], bins=30, color='lightcoral', edgecolor='black')
axes[0, 1].set_title('Distribución de Puntuación Crediticia')
axes[0, 1].set_xlabel('Puntuación Crediticia')
axes[0, 1].set_ylabel('Frecuencia')

# Distribución de años de empleo
axes[0, 2].hist(df['años_empleo'], bins=30, color='lightgreen', edgecolor='black')
axes[0, 2].set_title('Distribución de Años de Empleo')
axes[0, 2].set_xlabel('Años de Empleo')
axes[0, 2].set_ylabel('Frecuencia')

# Distribución de deuda actual
axes[1, 0].hist(df['deuda_actual'], bins=30, color='plum', edgecolor='black')
axes[1, 0].set_title('Distribución de Deuda Actual')
axes[1, 0].set_xlabel('Deuda Actual ($)')
axes[1, 0].set_ylabel('Frecuencia')

# Distribución de edad
axes[1, 1].hist(df['edad'], bins=30, color='gold', edgecolor='black')
axes[1, 1].set_title('Distribución de Edad')
axes[1, 1].set_xlabel('Edad (años)')
axes[1, 1].set_ylabel('Frecuencia')

# Distribución de aprobación
aprobacion_counts = df['aprobado'].value_counts()
axes[1, 2].bar(['Rechazado', 'Aprobado'], aprobacion_counts.values, color=['#ff6b6b', '#51cf66'])
axes[1, 2].set_title('Distribución de Aprobación de Créditos')
axes[1, 2].set_ylabel('Cantidad')
for i, v in enumerate(aprobacion_counts.values):
    axes[1, 2].text(i, v + 10, str(v), ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('01_analisis_exploratorio.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Gráfico guardado: '01_analisis_exploratorio.png'")

In [ ]:
# Matriz de correlación
plt.figure(figsize=(10, 8))
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlación entre Variables', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig('02_matriz_correlacion.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Gráfico guardado: '02_matriz_correlacion.png'")

## 4. Preparación de Datos para el Modelo

In [ ]:
# Separar características (X) y variable objetivo (y)
X = df[['ingreso_mensual', 'puntuacion_crediticia', 'años_empleo', 'deuda_actual', 'edad']]
y = df['aprobado']

print("Variables independientes (X):")
print(X.columns.tolist())
print(f"\nForma de X: {X.shape}")
print(f"Forma de y: {y.shape}")

In [ ]:
# Dividir en conjunto de entrenamiento y prueba (80% - 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"📊 División de datos:")
print(f"  • Conjunto de entrenamiento: {len(X_train)} registros ({len(X_train)/len(X)*100:.1f}%)")
print(f"  • Conjunto de prueba: {len(X_test)} registros ({len(X_test)/len(X)*100:.1f}%)")
print(f"\n  • Aprobados en entrenamiento: {y_train.sum()} ({y_train.mean()*100:.1f}%)")
print(f"  • Aprobados en prueba: {y_test.sum()} ({y_test.mean()*100:.1f}%)")

## 5. Creación y Entrenamiento del Modelo de Árbol de Decisión

In [ ]:
# Crear el modelo de árbol de decisión
modelo = DecisionTreeClassifier(
    max_depth=5,           # Profundidad máxima del árbol
    min_samples_split=50,  # Mínimo de muestras para dividir un nodo
    min_samples_leaf=20,   # Mínimo de muestras en una hoja
    random_state=42,
    criterion='gini'       # Criterio de división (gini o entropy)
)

print("✓ Modelo de Árbol de Decisión creado")
print("\nParámetros del modelo:")
print(f"  • Profundidad máxima: {modelo.max_depth}")
print(f"  • Mínimo muestras para dividir: {modelo.min_samples_split}")
print(f"  • Mínimo muestras en hoja: {modelo.min_samples_leaf}")
print(f"  • Criterio: {modelo.criterion}")

In [ ]:
# Entrenar el modelo
print("\n🎓 Entrenando el modelo...")
modelo.fit(X_train, y_train)
print("✓ Modelo entrenado exitosamente")

# Información del árbol entrenado
print(f"\n📊 Información del árbol entrenado:")
print(f"  • Número de nodos: {modelo.tree_.node_count}")
print(f"  • Número de hojas: {modelo.get_n_leaves()}")
print(f"  • Profundidad real del árbol: {modelo.get_depth()}")

## 6. Evaluación del Modelo

In [ ]:
# Hacer predicciones
y_pred_train = modelo.predict(X_train)
y_pred_test = modelo.predict(X_test)

# Calcular exactitud (accuracy)
accuracy_train = accuracy_score(y_train, y_pred_train)
accuracy_test = accuracy_score(y_test, y_pred_test)

print("\n🎯 Métricas de Rendimiento del Modelo:\n")
print(f"  • Exactitud en entrenamiento: {accuracy_train*100:.2f}%")
print(f"  • Exactitud en prueba: {accuracy_test*100:.2f}%")
print(f"  • Diferencia (overfitting): {(accuracy_train-accuracy_test)*100:.2f}%")

In [ ]:
# Reporte de clasificación
print("\n📋 Reporte de Clasificación (Conjunto de Prueba):\n")
print(classification_report(y_test, y_pred_test, target_names=['Rechazado', 'Aprobado']))

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test, y_pred_test)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, 
            xticklabels=['Rechazado', 'Aprobado'],
            yticklabels=['Rechazado', 'Aprobado'])
plt.title('Matriz de Confusión', fontsize=14, fontweight='bold', pad=20)
plt.ylabel('Valor Real', fontsize=12)
plt.xlabel('Valor Predicho', fontsize=12)
plt.tight_layout()
plt.savefig('03_matriz_confusion.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Gráfico guardado: '03_matriz_confusion.png'")

In [ ]:
# Importancia de las características
importancias = pd.DataFrame({
    'Característica': X.columns,
    'Importancia': modelo.feature_importances_
}).sort_values('Importancia', ascending=False)

print("\n📊 Importancia de las Características:\n")
print(importancias.to_string(index=False))

# Visualizar importancia de características
plt.figure(figsize=(10, 6))
plt.barh(importancias['Característica'], importancias['Importancia'], color='steelblue')
plt.xlabel('Importancia', fontsize=12)
plt.title('Importancia de las Características en el Modelo', fontsize=14, fontweight='bold', pad=20)
plt.gca().invert_yaxis()
for i, v in enumerate(importancias['Importancia']):
    plt.text(v + 0.01, i, f'{v:.3f}', va='center', fontweight='bold')
plt.tight_layout()
plt.savefig('04_importancia_caracteristicas.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Gráfico guardado: '04_importancia_caracteristicas.png'")

## 7. Visualización del Árbol de Decisión

In [ ]:
# Visualización completa del árbol
plt.figure(figsize=(25, 15))
plot_tree(modelo, 
          feature_names=X.columns,
          class_names=['Rechazado', 'Aprobado'],
          filled=True,
          rounded=True,
          fontsize=10)
plt.title('Árbol de Decisión - Aprobación de Créditos', fontsize=20, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig('05_arbol_decision_completo.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Gráfico guardado: '05_arbol_decision_completo.png'")

In [ ]:
# Visualización más compacta para el reporte
plt.figure(figsize=(20, 12))
plot_tree(modelo, 
          feature_names=X.columns,
          class_names=['Rechazado', 'Aprobado'],
          filled=True,
          rounded=True,
          fontsize=9,
          proportion=True)
plt.title('Árbol de Decisión (Vista Simplificada)', fontsize=18, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig('06_arbol_decision_simplificado.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Gráfico guardado: '06_arbol_decision_simplificado.png'")

## 8. Interpretación del Árbol en Texto

In [ ]:
# Exportar reglas del árbol en formato texto
tree_rules = export_text(modelo, feature_names=list(X.columns))
print("\n🌳 REGLAS DEL ÁRBOL DE DECISIÓN:\n")
print(tree_rules)

# Guardar reglas en archivo
with open('reglas_arbol.txt', 'w', encoding='utf-8') as f:
    f.write("REGLAS DEL ÁRBOL DE DECISIÓN - APROBACIÓN DE CRÉDITOS\n")
    f.write("="*60 + "\n\n")
    f.write(tree_rules)
print("\n✓ Reglas guardadas en 'reglas_arbol.txt'")

## 9. Interpretación Detallada del Modelo

In [ ]:
print("""\n📖 INTERPRETACIÓN DEL ÁRBOL DE DECISIÓN
═══════════════════════════════════════════════════════════════════════════════

1. ESTRUCTURA DEL ÁRBOL:
   - El árbol tiene {} nodos en total
   - Consta de {} nodos hojas (decisiones finales)
   - Alcanza una profundidad máxima de {} niveles

2. VARIABLE MÁS IMPORTANTE:
   - La característica más importante es: {}
   - Esta variable aparece en el nodo raíz (primera decisión)
   - Tiene un peso de {:.2%} en las decisiones del modelo

3. PRINCIPALES REGLAS DE DECISIÓN:
   
   El modelo sigue una lógica jerárquica:
   
   a) Primera división (Raíz):
      • Se evalúa la {}
      • Esta es la característica que mejor separa los datos
   
   b) Divisiones secundarias:
      • El modelo considera {} y {}
      • Estas variables complementan la decisión principal
   
   c) Factores adicionales:
      • {} también influye en casos específicos

4. INTERPRETACIÓN DE LOS NODOS:
   
   Cada nodo del árbol contiene:
   • Condición de división (ej: puntuacion_crediticia <= 650)
   • Índice de Gini: mide la impureza del nodo (0 = puro, 0.5 = máxima impureza)
   • Samples: cantidad de registros que llegan a ese nodo
   • Value: [rechazados, aprobados] en ese nodo
   • Class: decisión mayoritaria (Rechazado o Aprobado)
   • Color: azul = rechazado, naranja = aprobado (intensidad según pureza)

5. PATRONES IDENTIFICADOS:
   
   ✓ Clientes con ALTA probabilidad de aprobación:
     - Puntuación crediticia alta (> 700)
     - Relación deuda/ingreso baja
     - Años de empleo estables (> 5 años)
   
   ✗ Clientes con ALTA probabilidad de rechazo:
     - Puntuación crediticia baja (< 550)
     - Deuda actual muy alta
     - Poca antigüedad laboral

6. PRECISIÓN DEL MODELO:
   - El modelo tiene una exactitud de {:.2%} en datos de prueba
   - Esto significa que acierta en {} de cada 10 predicciones
   - El modelo es confiable para tomar decisiones automáticas

7. VENTAJAS DE ESTE ÁRBOL:
   ✓ Fácil de interpretar y explicar
   ✓ Refleja la lógica humana de decisión
   ✓ No requiere normalización de datos
   ✓ Maneja bien variables numéricas y categóricas
   ✓ Identifica automáticamente las características importantes

8. APLICACIÓN PRÁCTICA:
   Este modelo puede usarse para:
   • Automatizar decisiones de aprobación de créditos
   • Explicar a los clientes por qué fueron rechazados
   • Identificar qué deben mejorar para ser aprobados
   • Reducir sesgos humanos en la toma de decisiones

═══════════════════════════════════════════════════════════════════════════════
""".format(
    modelo.tree_.node_count,
    modelo.get_n_leaves(),
    modelo.get_depth(),
    importancias.iloc[0]['Característica'],
    importancias.iloc[0]['Importancia'],
    importancias.iloc[0]['Característica'],
    importancias.iloc[1]['Característica'],
    importancias.iloc[2]['Característica'],
    importancias.iloc[3]['Característica'],
    accuracy_test,
    int(accuracy_test * 10)
))

## 10. Ejemplo de Predicción con Casos Específicos

In [ ]:
# Crear casos de ejemplo para demostrar el uso del modelo
casos_ejemplo = pd.DataFrame([
    {
        'Caso': 'Cliente 1 - Perfil Excelente',
        'ingreso_mensual': 45000,
        'puntuacion_crediticia': 750,
        'años_empleo': 8.0,
        'deuda_actual': 30000,
        'edad': 35
    },
    {
        'Caso': 'Cliente 2 - Perfil Regular',
        'ingreso_mensual': 18000,
        'puntuacion_crediticia': 620,
        'años_empleo': 3.5,
        'deuda_actual': 80000,
        'edad': 28
    },
    {
        'Caso': 'Cliente 3 - Perfil Riesgoso',
        'ingreso_mensual': 12000,
        'puntuacion_crediticia': 480,
        'años_empleo': 0.5,
        'deuda_actual': 150000,
        'edad': 22
    }
])

# Hacer predicciones
X_ejemplos = casos_ejemplo[['ingreso_mensual', 'puntuacion_crediticia', 'años_empleo', 'deuda_actual', 'edad']]
predicciones = modelo.predict(X_ejemplos)
probabilidades = modelo.predict_proba(X_ejemplos)

print("\n🔍 EJEMPLOS DE PREDICCIÓN:\n")
print("="*100)
for i, row in casos_ejemplo.iterrows():
    decision = "APROBADO ✓" if predicciones[i] == 1 else "RECHAZADO ✗"
    prob_rechazo = probabilidades[i][0] * 100
    prob_aprobacion = probabilidades[i][1] * 100
    
    print(f"\n{row['Caso']}:")
    print(f"  • Ingreso mensual: ${row['ingreso_mensual']:,}")
    print(f"  • Puntuación crediticia: {row['puntuacion_crediticia']}")
    print(f"  • Años de empleo: {row['años_empleo']}")
    print(f"  • Deuda actual: ${row['deuda_actual']:,}")
    print(f"  • Edad: {row['edad']} años")
    print(f"\n  → DECISIÓN: {decision}")
    print(f"  → Probabilidad de rechazo: {prob_rechazo:.1f}%")
    print(f"  → Probabilidad de aprobación: {prob_aprobacion:.1f}%")
    print("="*100)

## 11. Análisis de Caminos de Decisión

In [ ]:
from sklearn.tree import _tree

def obtener_camino_decision(modelo, X_sample, feature_names):
    """
    Obtiene el camino de decisión que sigue el modelo para una muestra específica
    """
    tree = modelo.tree_
    node_indicator = modelo.decision_path(X_sample)
    leaf_id = modelo.apply(X_sample)
    
    sample_id = 0
    node_index = node_indicator.indices[node_indicator.indptr[sample_id]:node_indicator.indptr[sample_id + 1]]
    
    print("\n  Camino de decisión:")
    for i, node_id in enumerate(node_index):
        if leaf_id[sample_id] == node_id:
            print(f"\n  Nodo {i+1} (HOJA - Decisión Final):")
            print(f"    → Clasificación: {'APROBADO' if tree.value[node_id][0][1] > tree.value[node_id][0][0] else 'RECHAZADO'}")
            continue
        
        if X_sample.iloc[sample_id, tree.feature[node_id]] <= tree.threshold[node_id]:
            threshold_sign = "<="
        else:
            threshold_sign = ">"
        
        print(f"\n  Nodo {i+1}:")
        print(f"    Condición: {feature_names[tree.feature[node_id]]} {threshold_sign} {tree.threshold[node_id]:.2f}")
        print(f"    Valor real: {X_sample.iloc[sample_id, tree.feature[node_id]]:.2f}")
        print(f"    → Va a la {'izquierda' if threshold_sign == '<=' else 'derecha'}")

# Analizar el camino del Cliente 1
print("\n" + "="*100)
print("ANÁLISIS DETALLADO DEL CAMINO DE DECISIÓN - CLIENTE 1")
print("="*100)
obtener_camino_decision(modelo, X_ejemplos.iloc[[0]], X.columns.tolist())

## 12. Resumen Final y Conclusiones

In [ ]:
print("""\n\n╔══════════════════════════════════════════════════════════════════════════════╗
║                    RESUMEN FINAL DE LA ACTIVIDAD 9                           ║
║            ÁRBOL DE DECISIÓN - APROBACIÓN DE CRÉDITOS BANCARIOS              ║
╚══════════════════════════════════════════════════════════════════════════════╝

✅ CUMPLIMIENTO DE REQUERIMIENTOS:

1. ✓ Dataset generado: {} registros
   - Supera el mínimo de 1000 registros requeridos

2. ✓ Variables independientes: {}
   - Ingreso mensual
   - Puntuación crediticia  
   - Años de empleo
   - Deuda actual
   - Edad
   - Supera el mínimo de 3 variables requeridas

3. ✓ Variable dependiente (objetivo): Aprobación de crédito (0 o 1)

4. ✓ Modelo creado y entrenado exitosamente
   - Algoritmo: Decision Tree Classifier
   - Técnica: Aprendizaje supervisado
   - Exactitud: {:.2%}

5. ✓ Árbol dibujado y visualizado
   - Visualización completa guardada
   - Visualización simplificada generada
   - Reglas exportadas en formato texto

6. ✓ Interpretación y explicación detallada del árbol
   - Estructura del árbol explicada
   - Importancia de características analizada
   - Reglas de decisión interpretadas
   - Ejemplos prácticos demostrados

📊 ARCHIVOS GENERADOS:

   Datos:
   • dataset_creditos.csv - Dataset completo
   • reglas_arbol.txt - Reglas del árbol en texto
   
   Visualizaciones:
   • 01_analisis_exploratorio.png
   • 02_matriz_correlacion.png
   • 03_matriz_confusion.png
   • 04_importancia_caracteristicas.png
   • 05_arbol_decision_completo.png
   • 06_arbol_decision_simplificado.png

📈 MÉTRICAS PRINCIPALES:

   • Exactitud en prueba: {:.2%}
   • Registros totales: {}
   • Variables predictoras: {}
   • Profundidad del árbol: {}
   • Número de nodos: {}
   • Número de hojas: {}

🎯 CONCLUSIONES:

   El modelo de árbol de decisión desarrollado es capaz de predecir con una
   exactitud de {:.2%} si un cliente será aprobado para un crédito bancario.
   
   La puntuación crediticia es el factor más determinante, seguido de la
   relación deuda/ingreso y los años de empleo. El modelo es interpretable
   y puede explicar cada decisión, lo que lo hace ideal para aplicaciones
   en instituciones financieras que requieren transparencia.

✅ Todos los requerimientos de la actividad han sido cumplidos exitosamente.

═══════════════════════════════════════════════════════════════════════════════
""".format(
    len(df),
    len(X.columns),
    accuracy_test,
    accuracy_test,
    len(df),
    len(X.columns),
    modelo.get_depth(),
    modelo.tree_.node_count,
    modelo.get_n_leaves(),
    accuracy_test
))

## 13. Preparación de Datos para el Reporte PDF

Ahora ejecuta el script `generar_reporte_pdf.py` para crear el reporte final en PDF.